In [ ]:
aws_access_key = 'Access_key'
aws_secret_key = 'Secret_key'
region = 'us-east-1'

import boto3

def get_client(service):
    return boto3.client(service,
                        aws_access_key_id=aws_access_key,
                        aws_secret_access_key=aws_secret_key,
                        region_name=region)

In [ ]:
from botocore.exceptions import ClientError

def get_public_s3_buckets():
    s3 = get_client('s3')
    public_buckets = []

    for bucket in s3.list_buckets()['Buckets']:
        name = bucket['Name']
        try:
            acl = s3.get_bucket_acl(Bucket=name)
            for grant in acl['Grants']:
                if 'AllUsers' in str(grant['Grantee']):
                    public_buckets.append(name)
        except ClientError:
            pass
    return public_buckets

public_buckets = get_public_s3_buckets()
print("🚨 Public S3 Buckets:")
for b in public_buckets:
    print(f"- {b}")


In [ ]:
def get_open_security_groups():
    ec2 = get_client('ec2')
    open_sgs = []

    for sg in ec2.describe_security_groups()['SecurityGroups']:
        for rule in sg.get('IpPermissions', []):
            for ip_range in rule.get('IpRanges', []):
                if ip_range.get('CidrIp') == '0.0.0.0/0':
                    open_sgs.append(sg['GroupId'])
            for ipv6_range in rule.get('Ipv6Ranges', []):
                if ipv6_range.get('CidrIpv6') == '::/0':
                    open_sgs.append(sg['GroupId'])
    return list(set(open_sgs))

open_sgs = get_open_security_groups()
print("🚨 Open Security Groups:")
for sg in open_sgs:
    print(f"- {sg}")


In [ ]:
def get_over_permissive_iam_policies():
    iam = get_client('iam')
    risky_policies = []

    for role in iam.list_roles()['Roles']:
        role_name = role['RoleName']
        attached_policies = iam.list_attached_role_policies(RoleName=role_name)['AttachedPolicies']
        
        for policy in attached_policies:
            policy_arn = policy['PolicyArn']
            version_id = iam.get_policy(PolicyArn=policy_arn)['Policy']['DefaultVersionId']
            policy_doc = iam.get_policy_version(PolicyArn=policy_arn, VersionId=version_id)['PolicyVersion']['Document']

            statements = policy_doc['Statement']
            if not isinstance(statements, list):
                statements = [statements]

            for statement in statements:
                actions = statement.get('Action', [])
                resources = statement.get('Resource', [])
                if isinstance(actions, str): actions = [actions]
                if isinstance(resources, str): resources = [resources]

                if '*' in actions or '*' in resources:
                    risky_policies.append({
                        'RoleName': role_name,
                        'PolicyName': policy['PolicyName'],
                        'PolicyArn': policy_arn
                    })
    return risky_policies

risky_iam = get_over_permissive_iam_policies()
print("🚨 Over-Permissive IAM Policies:")
for policy in risky_iam:
    print(f"- Role: {policy['RoleName']}, Policy: {policy['PolicyName']}")


In [ ]:
def check_root_user_activity_and_mfa():
    iam = get_client('iam')
    root_user_status = {
        "RootUserUsedRecently": "Unknown",
        "RootMFAEnabled": "Unknown"
    }

    try:
        summary = iam.get_account_summary()['SummaryMap']

        if summary.get('AccountAccessKeysPresent', 0) > 0 or summary.get('AccountSigningCertificatesPresent', 0) > 0:
            root_user_status["RootUserUsedRecently"] = "Yes (access keys exist)"
        else:

            root_user_status["RootUserUsedRecently"] = "No access keys"

        mfa = summary.get('AccountMFAEnabled', 0)
        root_user_status["RootMFAEnabled"] = "Yes" if mfa else "No"

    except Exception as e:
        print(f"❌ Error checking root user status: {e}")

    return root_user_status

In [ ]:
root_status = check_root_user_activity_and_mfa()
print("🔐 Root User Security Check:")
for k, v in root_status.items():
    print(f"- {k}: {v}")

In [ ]:
summary_data = {
    "Check": [
        "Total S3 Buckets",
        "Public S3 Buckets",
        "Private S3 Buckets",
        "Total Security Groups",
        "Open Security Groups (0.0.0.0/0 or ::/0)",
        "Restricted Security Groups",
        "Over-Permissive IAM Policies (*:*)",
        "Root User Used Recently",
        "Root User MFA Enabled"
    ],
    "Count": [
        total_s3_buckets,
        num_public_buckets,
        num_private_buckets,
        total_sgs,
        num_open_sgs,
        num_restricted_sgs,
        num_risky_policies,
        root_status["RootUserUsedRecently"],
        root_status["RootMFAEnabled"]
    ]
}

summary_df = pd.DataFrame(summary_data)
summary_df
